<a href="https://colab.research.google.com/github/atstuyuki/mediapipe/blob/main/mediapipe_for_movie__Pose__20220208.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セットアップ

In [ ]:
# ライブラリーのインストール
!pip install numpy==1.19.3
!pip install mediapipe
#PySimpleGUIは後々組み込む
!pip install PySimpleGUI
# github からコードをコピー
#!git clone https://github.com/cedro3/mediapipe.git
#%cd mediapipe/

In [ ]:
# 初期設定 holistic とPoseを用意する　検出感度は0.5＞＞変更可能
import mediapipe as mp
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose
# Initialize MediaPipe Holistic.
holistic = mp_holistic.Holistic(
    static_image_mode=True, min_detection_confidence=0.2)
pose= mp_pose.Pose(
    static_image_mode=True, min_detection_confidence=0.2)
# Prepare DrawingSpec for drawing the face landmarks later.
mp_drawing = mp.solutions.drawing_utils 
drawing_spec = mp_drawing.DrawingSpec(color=[225,0,200],thickness=2, circle_radius=1) #color=[B,G,R]で指定,線の太さと〇の色も指定可能

In [ ]:
#3次元座標の角度計算の定義
def calculate_3D_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    #0がx座標　１がｙ座標　２がｚ座標
    #ベクトルを定義 長さと内積からコサインを計算
    vec_a=a-b
    vec_c=c-b
    len_vec_a=np.linalg.norm(vec_a)
    len_vec_c=np.linalg.norm(vec_c)
    inner_product=np.inner(vec_a, vec_c)
    cos=inner_product/(len_vec_a*len_vec_c)
    
    #角度（ラジアン）の計算
    rad=np.arccos(cos)
    
    #ラジアンから度数へ変換
    angle=np.rad2deg(rad)
        
    return angle

In [ ]:
#2次元からの角度計算の定義
def calculate_2D_angle(a,b,c):
  a=np.array(a)#firtst
  b=np.array(b)#second
  c=np.array(c)#third
  radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
  angle=np.abs(radians*180/np.pi)

  if angle>180.0:
    angle=360-angle
  
  return angle

In [ ]:
#手元の動画をgoogle colabにupload

from google.colab import files
import os
uploaded = files.upload()
#uploadedは辞書型なのでファイル名を取り出し、絶対パスを取得してfullpathという変数を当てる
for k in uploaded.keys():
  print('file name:',k)
fullpath=os.path.abspath(k)
print('fullpath:',fullpath)

# サンプルビデオの読み込み＋静止画に分解

In [ ]:
# video2images
import os
import shutil
import cv2

# 既にimagesフォルダーがあれば削除
if os.path.isdir('images'):
    shutil.rmtree('images')

os.makedirs('images', exist_ok=True)
 
def video_2_images(video_file= fullpath,   
                   image_dir='./images/', 
                   image_file='%s.png'):
 
    # Initial setting
    i = 0
    interval = 3#適宜調整
    length = 300
    
    cap = cv2.VideoCapture(video_file)
    while(cap.isOpened()):
        flag, frame = cap.read()  
        if flag == False:  
                break
        if i == length*interval:
                break
        if i % interval == 0:    
           cv2.imwrite(image_dir+image_file % str(int(i/interval)).zfill(6), frame)
        i += 1 
    cap.release()  
 
def main():
    video_2_images()
    
if __name__ == '__main__':
    main()

In [ ]:
#imageフォルダの最初の1枚目の画像を取得
firstimage =cv2.imread('./images/000000.png')
#imageのwidthとheightを取得
height, width, channels = firstimage.shape[:3]

In [ ]:
#pose_landmarkのリストを取得して配列に変換
import numpy as np
landmarklist=[]#空の配列を作成
for index in range(len(mp_holistic.PoseLandmark)):
    landmarklist.append(mp_holistic.PoseLandmark(index).name+'_x')
    landmarklist.append(mp_holistic.PoseLandmark(index).name+'_y')
    landmarklist.append(mp_holistic.PoseLandmark(index).name+'_z')
    landmarklist.append(mp_holistic.PoseLandmark(index).name+'_visibility')
#リストをarrayに変換
landmarklist=np.array(landmarklist).reshape(1,-1)
#landmarkshape の内容の確認
print(landmarklist.dtype)
print(landmarklist)
print(landmarklist.shape)

# MediaPipeで静止画を処理

In [38]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import glob
import time
#変数の初期化
data_land = np.zeros((0,132))
image_list =[0]#検出できた画像のリスト
nondetectimage_list= [0]#検出できなかった画像のリスト
# image file names to files in list format
files=[]
for name in sorted(glob.glob('./images/*.png')):
    files.append(name)

# Read images with OpenCV.
images = {name: cv2.imread(name) for name in files}
w = width             # 画像の横幅を取得
h = height            # 画像の縦幅を取得

for name, image in images.items():
  # Convert the BGR image to RGB and process it with MediaPipe Pose.
  if image is None:
      break
  results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  landmarks = results.pose_landmarks
  #get coordinate poseの33か所の3次元座標データと確信度(visibilisty)をすべて結合してCSVに変換
  data_land2 = np.zeros((1,4))

  if landmarks is None:#ランドマークを検出できなかったとき
        basename=os.path.basename(name)#ファイル名のみ取り出す
        filename=os.path.splitext(basename)[0]#さらに拡張子を除く
        nondetectimage_list.append(filename)# リストにファイル名番号を追加でリストに格納
        print(filename)# 検出できなかったファイル名の表示
        continue
  data_land2 = np.zeros((1,3))
  basename=os.path.basename(name)#ファイル名のみ取り出す
  filename=os.path.splitext(basename)[0]#さらに拡張子を除く
  image_list.append(filename)
  #get coordinate poseの33か所の3次元座標データと確信度(visibilisty)をすべて結合してCSVに変換  
  for x in range (0,33):
        data1 = results.pose_landmarks.landmark[x].x
        data2 = results.pose_landmarks.landmark[x].y
        data3 = results.pose_landmarks.landmark[x].z
        data4 = results.pose_landmarks.landmark[x].visibility
        keydata = np.hstack((data1,data2,data3,data4)).reshape(1,-1)
        data_land2 = np.hstack((data_land2,keydata))

  data_land2 = data_land2[:,3:]
  data_land = np.vstack((data_land,data_land2))
  final_data= np.vstack((landmarklist,data_land))

#cv2.imwrite(name, annotated_image)  
#savetxtではデータの型が合わないと保存的無いことがあるのでfmtで文字の型を指定
np.savetxt('keypoint_results.csv',final_data,delimiter = ',',fmt = '%s')


000007
000008
000009
000010
000011
000015
000016
000018
000019
000020
000021
000022
000023
000024
000025
000027
000031
000032
000033
000034
000035
000036
000037
000038
000039
000040
000041
000042
000043
000044
000046
000048
000049
000050
000051
000052
000053
000054
000055
000056
000057
000058


画像番号のリストと検出部位のデータを結合

In [40]:
  #検出できなかった画像、検出できた画像の番号をintデータに変換してリストに格納
  nondetectimage_list=[*map(int,nondetectimage_list)]#map関数でstr→intに変換したうえでmapオブジェクトを＊でリストに変換
  image_list=[*map(int,image_list)]
  print('nondetectimage_list:',nondetectimage_list)
  print('detectimage_list:',image_list)

nondetectimage_list: [0, 7, 8, 9, 10, 11, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 27, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58]
detectimage_list: [0, 0, 1, 2, 3, 4, 5, 6, 12, 13, 14, 17, 26, 28, 29, 30, 45, 47]


In [52]:
#detectimage_listは列ベクトルに変換してfinal dataと結合する
image_array=np.array(image_list) #listをarrayに変換
image_array_T=image_array.T #転置行列に変換
print('image_array_T.shape:',image_array_T.shape)
print('final_data.shape:',final_data.shape)

image_array_T.shape: (18, 1)
final_data.shape: (18, 132)


In [59]:
#検出できた画像番号のarrayと座標のデータを水平結合
image_final_data=np.hstack((image_array_T,final_data))
print('conbined image number and final data shape:',image_final_data.shape)

conbined image number and final data shape: (18, 133)


In [66]:
#検出できなかった画像nondetectimage_listをarray→転置行列変換
nondetectimage_array=np.array(nondetectimage_list) #listをarrayに変換
nondetectimage_array_T=nondetectimage_array.T.reshape(-1,1) #転置行列に変換
print('nondetectimage_array_T.shape:',nondetectimage_array_T.shape)

nondetectimage_array_T.shape: (43, 1)


In [67]:
#行が検出できなかった画像の数+1　で列が座標データ数のzero行列を作成
nondetect_landmark=np.zeros((len(nondetectimage_list), 132))
nondetect_image_landmark=np.hstack((nondetectimage_array_T,nondetect_landmark))#画像番号とゼロ行列を結合
print('nondetect_landmark_shape:',nondetect_image_landmark.shape)

nondetect_landmark_shape: (43, 133)


In [76]:
#検出・非検出の画像番号および座標データを垂直結合
import pandas as pd
all_image_landmark=np.vstack((image_final_data,nondetect_image_landmark))
print('all image landmark shape:',all_image_landmark.shape)#形の確認
dataframe=pd.DataFrame(all_image_landmark)#ndarrayをpandasのDataFrameに変換
dataframe.head(20)#dataframeの最初の20行の表示

all image landmark shape: (61, 133)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132
0,0,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_EYE_INNER_x,LEFT_EYE_INNER_y,LEFT_EYE_INNER_z,LEFT_EYE_INNER_visibility,LEFT_EYE_x,LEFT_EYE_y,LEFT_EYE_z,LEFT_EYE_visibility,LEFT_EYE_OUTER_x,LEFT_EYE_OUTER_y,LEFT_EYE_OUTER_z,LEFT_EYE_OUTER_visibility,RIGHT_EYE_INNER_x,RIGHT_EYE_INNER_y,RIGHT_EYE_INNER_z,RIGHT_EYE_INNER_visibility,RIGHT_EYE_x,RIGHT_EYE_y,RIGHT_EYE_z,RIGHT_EYE_visibility,RIGHT_EYE_OUTER_x,RIGHT_EYE_OUTER_y,RIGHT_EYE_OUTER_z,RIGHT_EYE_OUTER_visibility,LEFT_EAR_x,LEFT_EAR_y,LEFT_EAR_z,LEFT_EAR_visibility,RIGHT_EAR_x,RIGHT_EAR_y,RIGHT_EAR_z,RIGHT_EAR_visibility,MOUTH_LEFT_x,MOUTH_LEFT_y,MOUTH_LEFT_z,...,LEFT_HIP_x,LEFT_HIP_y,LEFT_HIP_z,LEFT_HIP_visibility,RIGHT_HIP_x,RIGHT_HIP_y,RIGHT_HIP_z,RIGHT_HIP_visibility,LEFT_KNEE_x,LEFT_KNEE_y,LEFT_KNEE_z,LEFT_KNEE_visibility,RIGHT_KNEE_x,RIGHT_KNEE_y,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility,LEFT_HEEL_x,LEFT_HEEL_y,LEFT_HEEL_z,LEFT_HEEL_visibility,RIGHT_HEEL_x,RIGHT_HEEL_y,RIGHT_HEEL_z,RIGHT_HEEL_visibility,LEFT_FOOT_INDEX_x,LEFT_FOOT_INDEX_y,LEFT_FOOT_INDEX_z,LEFT_FOOT_INDEX_visibility,RIGHT_FOOT_INDEX_x,RIGHT_FOOT_INDEX_y,RIGHT_FOOT_INDEX_z,RIGHT_FOOT_INDEX_visibility
1,0,0.5305181741714478,0.35457706451416016,-0.12011449784040451,0.9982500672340393,0.5274416208267212,0.3508709669113159,-0.1165042370557785,0.9955947995185852,0.5259547829627991,0.350565642118454,-0.1165570542216301,0.9949654936790466,0.5244725346565247,0.350246787071228,-0.11663166433572769,0.9963023662567139,0.5308656692504883,0.35181427001953125,-0.14093933999538422,0.9976755976676941,0.5317636132240295,0.35222020745277405,-0.14097772538661957,0.9974460601806641,0.5328390002250671,0.35259854793548584,-0.1410476565361023,0.9979134202003479,0.5205549597740173,0.3522630035877228,-0.08623015135526657,0.9921390414237976,0.5331714153289795,0.35520318150520325,-0.19651055335998535,0.9971357583999634,0.529472291469574,0.3578746020793915,-0.1055184155702591,...,0.5512946248054504,0.42949098348617554,0.06991919130086899,0.9985613226890564,0.5459963083267212,0.43207329511642456,-0.06996653974056244,0.9979648590087891,0.5940238833427429,0.46088433265686035,0.06412362307310104,0.7737487554550171,0.5930156111717224,0.46236199140548706,-0.07997528463602066,0.990712583065033,0.6289414167404175,0.49801474809646606,0.1693766862154007,0.8998770713806152,0.6272684335708618,0.4965594410896301,0.014999184757471085,0.9873787760734558,0.6277427673339844,0.5060112476348877,0.17671914398670197,0.817541778087616,0.6276689171791077,0.5024541616439819,0.020531361922621727,0.9406346678733826,0.6530623435974121,0.506414532661438,0.11509139090776443,0.922817051410675,0.6528820991516113,0.508348822593689,-0.054045259952545166,0.974804162979126
2,1,0.5515859127044678,0.3549952208995819,-0.17195868492126465,0.9902732968330383,0.5478595495223999,0.3511195182800293,-0.1667328178882599,0.9797163605690002,0.5466912984848022,0.3509421646595001,-0.1668100208044052,0.9792858958244324,0.5455809235572815,0.3507624864578247,-0.16691620647907257,0.9848932027816772,0.5504319667816162,0.35175657272338867,-0.19135524332523346,0.9867313504219055,0.5510522127151489,0.35206282138824463,-0.19139644503593445,0.9861300587654114,0.5516902804374695,0.3523693382740021,-0.19144965708255768,0.9881006479263306,0.5410249829292297,0.35243332386016846,-0.1207023411989212,0.9721212387084961,0.5498059988021851,0.35447174310684204,-0.23055757582187653,0.9825733304023743,0.5505333542823792,0.35798609256744385,-0.1505901962518692,...,0.5683391690254211,0.42483097314834595,0.06400302797555923,0.9979335069656372,0.5543175339698792,0.4316553473472595,-0.06395049393177032,0.9970927238464355,0.62047

In [ ]:
#CSVデータと画像の枚数が一致しているか確認
print(final_data.shape)
print(len(images))

In [64]:
nondetect_landmark.shape

(43, 132)

# 処理した画像をmp4動画に変換


In [ ]:
# 既に output.mp4 があれば削除
import os
if os.path.exists('./output.mp4'):
   os.remove('./output.mp4')

! ffmpeg -r 10 -i images/%6d.png\
               -vcodec libx264 -pix_fmt yuv420p output.mp4

In [ ]:
# --- mp4動画の再生 ---
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="85%" height="85%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

＊作成したmp4動画は、左側のファイル欄にある　./mediapipe/output.mp4 を右クリックするとダウンロードできます。

# 処理した画像をgif動画に変換


In [ ]:
# images2gif
from PIL import Image
import glob
 
files = sorted(glob.glob('./images/*.png'))
images = list(map(lambda file: Image.open(file), files))
images[0].save('./out.gif', save_all=True, 
               append_images=images[1:], 
               duration=100, loop=0)

In [ ]:
# サイズの大きいgifは再生しないで下さい（クラッシュするため）。
# display gif
from IPython.display import Image
Image('./out.gif', format='png')

＊作成したgif動画は、左側のファイル欄にある ./mediapipe/out.gif を右クリックするとダウンロードできます。